In [66]:

import pandas as pd
import datetime as dt 
import zipfile
import os
from dateutil import parser

In [67]:
def gender_dt(D):
    if (D == 1):
        return 'Male'
    elif (D== 2) :
        return 'Female'
    else :
        return 'Unknown'

In [68]:
def bikedata_read(file):
    bikedata=pd.read_csv(file,low_memory=False , header=0, names=["tripduration","starttime","stoptime","start station id","start station name",
                                   "start station latitude","start station longitude","end station id","end station name",
                                   "end station latitude","end station longitude","bikeid","usertype","birth year","gender"])

   
    bikedata['starttime']= bikedata.starttime.apply(lambda d:parser.parse(d))


    bikedata['startdate']=bikedata.starttime.apply(lambda d: d.date())
    bikedata['start_time']=bikedata.starttime.apply(lambda d: d.hour)


    bikedata['stoptime']= bikedata.stoptime.apply(lambda d:parser.parse(d))

    bikedata['gender']=bikedata.gender.apply(lambda d : gender_dt(d))


    if bikedata['birth year'].dtype=="object" :
        bikedata['birth year']=bikedata['birth year'].apply(lambda d: 1800 if d=='\\N' else int(d))
    else :
        bikedata['birth year']. fillna(1800,inplace=True)

    bikedata['age']=bikedata['birth year'].apply(lambda d: (dt.datetime.now().year- d))

    

    bikedata['start station id'].fillna(0, inplace=True)
    bikedata['end station id'].fillna(0, inplace=True)
    bikedata['start station id']=bikedata['start station id'].astype(int)
    bikedata['end station id']= bikedata['end station id'].astype(int)
    bikedata['start station name'].fillna('UNKNOWN', inplace=True)
    bikedata['end station name'].fillna('UNKNOWN', inplace=True)


    age_labels =["15-25","25-40","40-55","55-70","70+"]
    age_bin=[0,26,41,56,71,150]
    bikedata["ageGroup"]=pd.cut(bikedata['age'],age_bin,labels=age_labels)



    hours_labels = ["midnight-5AM","5AM - 9AM","10AM -1PM","1PM - 4PM","4PM - 7PM","7PM - midnight"]
    hours_bin=[-1,6,10,14,17,20,24]
    bikedata["hrGroup"]=pd.cut(bikedata['start_time'],hours_bin,labels=hours_labels)
    bikedata['tripduration']=bikedata['tripduration']/3600
    bikedata['ride_cnt'] =bikedata.index 

#     bikedata_grp=bikedata[['start station id','start station name','end station id','end station name','bikeid','gender','startdate','ageGroup','hrGroup','tripduration','usertype','ride_cnt']].\
#     groupby(['start station id','start station name','end station id','end station name','bikeid','gender','startdate','ageGroup','usertype','hrGroup']).agg({'tripduration': 'sum' ,'ride_cnt': 'count'})

#     bikedata_grp.reset_index(inplace=True)
    return bikedata

In [69]:
files= os.listdir("C:/users/lalit/learnpanda/citibike")
#print(os.path.join("C:/users/lalit/learnpanda/citibike/",file))
big_frame=pd.DataFrame()
for file in files:
    #print(os.path.join("C:/users/lalit/learnpanda/citibike/",file))
    df1=bikedata_read(os.path.join("C:/users/lalit/learnpanda/citibike/",file))
    #df1.head()
    big_frame=pd.concat([big_frame,df1])

In [70]:
bikedata_grp=big_frame[['start station id','start station name','end station id','end station name','bikeid','gender','startdate','ageGroup','hrGroup','tripduration','usertype','ride_cnt']].\
     groupby(['start station id','start station name','end station id','end station name','bikeid','gender','startdate','ageGroup','usertype','hrGroup']).agg({'tripduration': 'sum' ,'ride_cnt': 'count'})
bikedata_grp.reset_index(inplace=True)

In [71]:
bikedata_grp.to_csv("C:\DataAnalytics\Bikeyeartrends.csv")

In [72]:
path_df1=big_frame[['start station id','start station name','start station latitude','start station longitude']].copy()

In [29]:
big_frame.head()

""


In [80]:
path_df1['year']=big_frame.starttime.apply(lambda d: d.year)

In [81]:
path_df1['path']=big_frame['start station id'].astype(str)+ '__'+ big_frame['end station id'].astype(str) + '_'+path_df1['year'].astype(str)

In [82]:
path_df1.columns = ['Stationid','Station_name', 'latitude',
      'longitude','year','path']
path_df1['type']='Start'

ValueError: Length mismatch: Expected axis has 7 elements, new values have 6 elements

In [ ]:
path_df1

In [50]:
path_df1['cnt']=path_df1.index

In [51]:
path_df1

,Stationid,Station_name,latitude,longitude,year,path,type,cnt
0,3494,E 115 St & Lexington Ave,40.797911,-73.942300,2019,3494__3501_2019,Start,0
1,438,St Marks Pl & 1 Ave,40.727791,-73.985649,2019,438__236_2019,Start,1
2,3571,Bedford Ave & Bergen St,40.676368,-73.952918,2019,3571__3549_2019,Start,2
3,167,E 39 St & 3 Ave,40.748901,-73.976049,2019,167__477_2019,Start,3
4,3458,W 55 St & 6 Ave,40.763094,-73.978350,2019,3458__3443_2019,Start,4
5,3078,Broadway & Roebling St,40.709248,-73.960631,2019,3078__3016_2019,Start,5
6,411,E 6 St & Avenue D,40.722281,-73.976687,2019,411__317_2019,Start,6
7,3628,Lenox Ave & W 117 St,40.802557,-73.949078,2019,3628__3506_2019,Start,7
8,293,Lafayette St & E 8 St,40.730207,-73.991026,2019,293__383_2019,Start,8
9,380,W 4 St & 7 Ave S,40.734011,-74.002939,2019,380__3263_2019,Start,9


In [83]:
path_df2=big_frame[['end station id','end station name','end station latitude','end station longitude']].copy()

In [84]:
path_df2['year']=big_frame.starttime.apply(lambda d: d.year)

In [85]:
path_df2['path']=big_frame['start station id'].astype(str)+ '__'+ big_frame['end station id'].astype(str)+'_'+path_df2['year'].astype(str)

In [86]:
path_df2.columns = ['Stationid','Station_name', 'latitude',
      'longitude','year','path']
path_df2['type']='Destination'
path_df2['cnt']=path_df1.index

In [87]:
path_final=path_df1.append(path_df2,ignore_index=True)


C:\Users\lalit\Anaconda3\envs\pythondata\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [88]:
path_final =path_final.groupby(['path','Stationid','Station_name','latitude','longitude','year','type']).agg('count')

In [89]:
path_final.reset_index(inplace=True)

In [90]:
path_final=path_final[path_final.path != '0__0']

In [91]:
path_final

,path,Stationid,Station_name,latitude,longitude,year,type,cnt
0,0__0_2019,0,UNKNOWN,40.842000,-73.881000,2019,Start,0
1,0__0_2019,0,UNKNOWN,40.845000,-73.902000,2019,Start,0
2,0__0_2019,0,UNKNOWN,40.857000,-73.890000,2019,Destination,2
3,0__0_2019,0,UNKNOWN,40.857000,-73.890000,2019,Start,0
4,0__0_2019,0,UNKNOWN,40.857000,-73.887000,2019,Destination,1
5,0__0_2019,0,UNKNOWN,40.857000,-73.881000,2019,Destination,1
6,0__0_2019,0,UNKNOWN,40.857000,-73.881000,2019,Start,0
7,0__0_2019,0,UNKNOWN,40.860000,-73.890000,2019,Destination,1
8,0__0_2019,0,UNKNOWN,40.860000,-73.884000,2019,Destination,1
9,0__0_2019,0,UNKNOWN,40.860000,-73.884000,2019,Start,0


In [92]:
path_final.to_csv("C:\DataAnalytics\path_yeartrends_2019.csv")